In [251]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from IPython.display import display, HTML

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
symbols = '!@#$%^&*()_+-=[]{}|;:,.<>?'
numbers = '0123456789'
mapping = {}

themes = {
    "falcon": {
        "bg": "#FFF5E1",
        "fg": "#3B1F1F",
        "button_bg": "#FF6F00",
        "button_fg": "#3B1F1F",
        "selected_bg": "#C0C0C0",
        "selected_fg": "white"
    },
    "light": {
        "bg": "white",
        "fg": "black",
        "button_bg": "#f0f0f0",
        "button_fg": "black",
        "selected_bg": "#FFD700",
        "selected_fg": "red"
    },
    "dark": {
        "bg": "#2e2e2e",
        "fg": "white",
        "button_bg": "#444",
        "button_fg": "white",
        "selected_bg": "#C0C0C0",
        "selected_fg": "red"
    }
}

current_theme = "light"
map_buttons = []
buttons = []
selected_btn = [None]

In [252]:
def open_instruction_popup():
    popup = tk.Toplevel(root)
    popup.title("Instructions")
    popup.geometry("400x300")

    instructions = """\
Welcome to your Subsitution Cipher Encoder!!!!
This tool is designed to help you encode phrases and words. 
Here is how you use this tool:
1. First, enter the text you want to encode.
2. Click on the letter in the top rows to select a source letter.
3. Click on a letter in the bottom rows to map it to the selected source.
You don't have to map everything, just what you want to change.
4. Click "Encode" to see your encoded message.
5. Click "Download Key" to save your substitution key as a PDF.

Extra: Use the theme button (💡) to cycle color themes.

Have fun using this tool!!!!
"""

    txt = scrolledtext.ScrolledText(popup, wrap=tk.WORD)
    txt.pack(expand=True, fill='both', padx=10, pady=10)
    txt.insert(tk.END, instructions)
    txt.config(state='disabled')  # make it read-only

    close_btn = tk.Button(popup, text="Close", command=popup.destroy)
    close_btn.pack(pady=5)



In [253]:
def map_letter(ltr):
    source = selected.get()
    if source:
        if ltr in mapping.values():
            messagebox.showerror("Error", "You already used that letter")
            update_mapping()
            return
        mapping[source] = ltr
        selected.set('')
        selected_btn[0] = None
        update_mapping()

def update_mapping():
    for btn in buttons:  # top row
        ltr = btn.letter
        mapped = mapping.get(ltr, '_')
        btn.config(text=f"{ltr}\n-> {mapped}")

        if ltr in mapping:
            btn.config(bg='green')
        else:
            if selected_btn[0] and btn == selected_btn[0]:
                btn.config(bg=themes[current_theme]["selected_bg"], fg=themes[current_theme]["button_fg"])
            else:
                btn.config(bg=themes[current_theme]["button_bg"], fg=themes[current_theme]["button_fg"])

    for btn in map_buttons:  # bottom row
        if btn.letter in mapping.values():
            btn.config(bg='red')
        else:
            btn.config(bg=themes[current_theme]["button_bg"], fg=themes[current_theme]["button_fg"])

def assign_letter(ltr):
    selected.set(ltr)
    if selected_btn[0]:
        selected_btn[0].config(bg=themes[current_theme]["button_bg"])
    for btn in buttons:
        if btn.letter == ltr:
            btn.config(bg=themes[current_theme]["selected_bg"])
            selected_btn[0] = btn
            break

def apply_theme():
    theme = themes[current_theme]
    root.configure(bg=theme["bg"])
    input_entry.configure(bg=theme["bg"], fg=theme["fg"], insertbackground=theme["fg"])
    
    for widget in root.winfo_children():
        if isinstance(widget, tk.Label) or isinstance(widget, tk.Button):
            widget.configure(bg=theme["button_bg"], fg=theme["button_fg"])

    for btn in buttons:
        if selected_btn[0] and btn == selected_btn[0]:
            btn.configure(bg=theme["selected_bg"], fg=theme["button_fg"])
        elif btn.letter in mapping:
            btn.configure(bg=theme["selected_bg"], fg=theme["selected_fg"])  # mapped source letter
        else:
             btn.configure(bg=theme["button_bg"], fg=theme["button_fg"])

    for btn in map_buttons:
        if btn.letter in mapping.values():
            btn.configure(bg='red', fg=theme["button_fg"])
        else:
            btn.configure(bg=theme["button_bg"], fg=theme["button_fg"])

def change_theme():
    global current_theme
    theme_keys = list(themes.keys())
    current_index = theme_keys.index(current_theme)
    next_index = (current_index + 1) % len(theme_keys)
    current_theme = theme_keys[next_index]
    apply_theme()

def show_encoded():
    plain_text = input_var.get()
    encoded = ""

    for char in plain_text:
        upper_char = char.upper()
        if upper_char in mapping:
            mapped = mapping[upper_char]
            if char.isupper():
                encoded += mapped.upper()
            else:
                encoded += mapped.lower()
        else:
            encoded += char

    messagebox.showinfo("Encoded Message", encoded)

def save_key_pdf():
    filename = "substitution_key.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    c.setFont("Helvetica", 14)
    c.drawString(100, 750, "Substitution Cipher Key")
    y = 720
    for k in sorted(mapping.keys()):
        c.drawString(100, y, f"{k} -> {mapping[k]}")
        y -= 20
        if y < 50:
            c.showPage()
            y = 750
    c.save()
    messagebox.showinfo("PDF Saved", f"Key saved as {filename}")


In [254]:
# GUI Setup
root = tk.Tk()
root.title("Substitution Cipher Mapper")

for i in range(30):
    root.grid_columnconfigure(i, weight=1, uniform="col")
for i in range(10):
    root.grid_rowconfigure(i, weight=1, uniform="row")

label = tk.Label(root, text="Welcome to your substitution cipher tool, please enter your desired text below", font=("Helvetica", 14))
label.grid(row=0, column=2, columnspan=22, pady=5)

# Instructions button next to label
instr_button = tk.Button(root, text="Show Instructions", command=open_instruction_popup)
instr_button.grid(row=0, column=23, columnspan=3, padx=5, pady=5, sticky="ew")

tk.Button(root, text="💡", command=change_theme, wraplength=80).grid(row=0, column=26, padx=5, pady=5, sticky="ew")

input_var = tk.StringVar()
input_entry = tk.Entry(root, textvariable=input_var, width=30)
input_entry.grid(row=1, column=0, columnspan=27, padx=5, pady=5)

selected = tk.StringVar()

# Alphabet buttons (row 3)
for i, ltr in enumerate(alphabet):
    btn = tk.Button(root, text=f"{ltr}\n->_", command=lambda l=ltr: assign_letter(l))
    btn.letter = ltr
    btn.grid(row=3, column=i, padx=2, pady=2, sticky="nsew")
    buttons.append(btn)

# Symbol buttons (row 4)
for i, ltr in enumerate(symbols):
    btn = tk.Button(root, text=f"{ltr}\n->_", command=lambda l=ltr: assign_letter(l))
    btn.letter = ltr
    btn.grid(row=4, column=i, padx=2, pady=2, sticky="nsew")
    buttons.append(btn)

# Number buttons (row 5)
for i, ltr in enumerate(numbers):
    btn = tk.Button(root, text=f"{ltr}\n->_", command=lambda l=ltr: assign_letter(l))
    btn.letter = ltr
    btn.grid(row=5, column=i, padx=2, pady=2, sticky="nsew")
    buttons.append(btn)

# Mapping buttons - alphabet (row 7)
for i, ltr in enumerate(alphabet):
    btn = tk.Button(root, text=ltr, command=lambda l=ltr: map_letter(l))
    btn.letter = ltr
    btn.grid(row=7, column=i, padx=1, pady=1, sticky="nsew")
    map_buttons.append(btn)

# Mapping buttons - symbols (row 8)
for i, ltr in enumerate(symbols):
    btn = tk.Button(root, text=ltr, command=lambda l=ltr: map_letter(l))
    btn.letter = ltr
    btn.grid(row=8, column=i, padx=1, pady=1, sticky="nsew")
    map_buttons.append(btn)

# Mapping buttons - numbers (row 9)
for i, ltr in enumerate(numbers):
    btn = tk.Button(root, text=ltr, command=lambda l=ltr: map_letter(l))
    btn.letter = ltr
    btn.grid(row=9, column=i, padx=1, pady=1, sticky="nsew")
    map_buttons.append(btn)

    
tk.Button(root, text="Encode", command=show_encoded).grid(row=9, column=0, columnspan=27, pady=5)
tk.Button(root, text="Download Key", command=save_key_pdf).grid(row=9, column=3, columnspan=27, pady=5)
tk.Label(root, text="Select Character").grid(row=2, column=0, columnspan=27, pady=5)
tk.Label(root, text="Map Character To...").grid(row=6, column=0, columnspan=27, pady=5)
apply_theme()
tk.mainloop()